# Example Predictor: XGBoost Predictor

This example contains basic functionality for training and evaluating a XGBoost predictor.

First, a training data set is created from historical case and npi data.

Second, a XGBoost model is trained to predict future cases from prior case data along with prior and future npi data.
The model is an off-the-shelf sklearn XGBoost model.

Third, a sample evaluation set is created, and the predictor is applied to this evaluation set to produce prediction results in the correct format.

## Training

In [1]:
import pickle

import numpy as np
import pandas as pd
import sklearn
import xgboost as xgb
from sklearn.model_selection import train_test_split, GridSearchCV
from xgboost import plot_importance

In [2]:
# Load historical data from URL
URL = 'https://raw.githubusercontent.com/OxCGRT/covid-policy-tracker/master/data/OxCGRT_latest.csv'
df = pd.read_csv(URL, 
                 parse_dates=['Date'],
                 encoding="ISO-8859-1",
                 error_bad_lines=False)

/home/darren.sargent/venv/covid-xprize/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (2,3) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
# For testing, restrict training data to that before a hypothetical predictor submission date
HYPOTHETICAL_SUBMISSION_DATE = np.datetime64("2020-07-31")
df = df[df.Date <= HYPOTHETICAL_SUBMISSION_DATE]

In [4]:
# Add RegionID column that combines CountryName and RegionName for easier manipulation of data
df['GeoID'] = df['CountryName'] + '__' + df['RegionName'].astype(str)

In [5]:
# Add new cases column
df['NewCases'] = df.groupby('GeoID').ConfirmedCases.diff().fillna(0)

In [6]:
# Keep only columns of interest
id_cols = ['CountryName',
           'RegionName',
           'GeoID',
           'Date']
cases_col = ['NewCases']
npi_cols = ['C1_School closing',
            'C2_Workplace closing',
            'C3_Cancel public events',
            'C4_Restrictions on gatherings',
            'C5_Close public transport',
            'C6_Stay at home requirements',
            'C7_Restrictions on internal movement',
            'C8_International travel controls',
            'H1_Public information campaigns',
            'H2_Testing policy',
            'H3_Contact tracing']
df = df[id_cols + cases_col + npi_cols]

In [7]:
# Fill any missing case values by interpolation and setting NaNs to 0
df.update(df.groupby('GeoID').NewCases.apply(
    lambda group: group.interpolate()).fillna(0))

In [8]:
# Fill any missing NPIs by assuming they are the same as previous day
for npi_col in npi_cols:
    df.update(df.groupby('GeoID')[npi_col].ffill().fillna(0))

In [9]:
df

,CountryName,RegionName,GeoID,Date,NewCases,C1_School closing,C2_Workplace closing,C3_Cancel public events,C4_Restrictions on gatherings,C5_Close public transport,C6_Stay at home requirements,C7_Restrictions on internal movement,C8_International travel controls,H1_Public information campaigns,H2_Testing policy,H3_Contact tracing
0,Aruba,NaN,Aruba__nan,2020-01-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Aruba,NaN,Aruba__nan,2020-01-02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Aruba,NaN,Aruba__nan,2020-01-03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Aruba,NaN,Aruba__nan,2020-01-04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Aruba,NaN,Aruba__nan,2020-01-05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64048,Zimbabwe,NaN,Zimbabwe__nan,2020-07-27,78.0,3.0,1.0,2.0,3.0,1.0,2.0,2.0,4.0,2.0,1.0,1.0
64049,Zimbabwe,NaN,Zimbabwe__nan,2020-07-28,192.0,3.0,1.0,2.0,3.0,1.0,2.0,2.0,4.0,2.0,1.0,1.0
64050,Zimbabwe,NaN,Zimbabwe__nan,2020-07-29,113.0,3.0,1.0,2.0,3.0,1.0,2.0,2.0,4.0,2.0,1.0,1.0
64051,Zimbabwe,NaN,Zimbabwe__nan,2020-07-30,62.0,3.0,1.0,2.0,3.0,1.0,2.0,2.0,4.0,2.0,1.0,1.0


In [10]:
# Next 2 cells: Functions for training one model for each day into the future we want to predict

In [11]:
# Create training data across all countries for predicting nb_days_ahead
def create_training_data(nb_days_ahead, nb_lookback_days, df, cases_col, npi_cols):
    X_samples = []
    y_samples = []
    geo_ids = df.GeoID.unique()
    for g in geo_ids:
        gdf = df[df.GeoID == g]
        all_case_data = np.array(gdf[cases_col])
        all_npi_data = np.array(gdf[npi_cols])

        # Create one sample for each day where we have enough data
        nb_total_days = len(gdf)
        for day  in range(nb_lookback_days, nb_total_days - nb_days_ahead):
            X_cases = all_case_data[day - nb_lookback_days:day ]

            X_npis = all_npi_data[day  - nb_lookback_days:day  + nb_days_ahead]

            # Flatten all input data so it fits XGBoost input format.
            X_sample = np.concatenate([X_cases.flatten(),
                                       X_npis.flatten()])
            y_sample = all_case_data[day  + nb_days_ahead]
            X_samples.append(X_sample)
            y_samples.append(y_sample)

    X_samples = np.array(X_samples)
    y_samples = np.array(y_samples).flatten()
    return X_samples, y_samples

In [12]:
# Compute mae
def mae(pred, true):
    return np.mean(np.abs(pred - true))

# Use grid search to optimize hyperparameters for XGBoost
# Typical result (used in create_and_train_xgb_model):
#
# {'colsample_bytree': 0.3, 'gamma': 5, 'learning_rate': 0.01, 'max_depth': 10, 'n_estimators': 1000, 
#  'reg_alpha': 50}
#
def search_hyperparams(X_samples, y_samples):
        # Split data into train and test sets
        X_train, X_test, y_train, y_test = train_test_split(X_samples,
                                                            y_samples,
                                                            test_size=0.2,
                                                            random_state=301)

        tuned_parameters = {
            'learning_rate': [0.1, 0.01, 0.001],
            'colsample_bytree': [0.3, 0.5, 0.8],
            'n_estimators': [1, 10, 100, 1000],
            'reg_alpha': [40, 50, 100],
            'max_depth': [3, 5, 10],
            'gamma': [0, 1, 5]
        }

        scores = ['neg_mean_absolute_error']

        for score in scores:
            print("# Tuning hyper-parameters for %s" % score)
            print()

            clf = GridSearchCV(
                 xgb.XGBRegressor(), tuned_parameters, scoring=score, verbose=2
            )
            clf.fit(X_train, y_train)

            print("Best parameters set found on development set:")
            print()
            print(clf.best_params_)
            print()
            print("Grid scores on development set:")
            print()
            means = clf.cv_results_['mean_test_score']
            stds = clf.cv_results_['std_test_score']
            for mean, std, params in zip(means, stds, clf.cv_results_['params']):
                print("%0.3f (+/-%0.03f) for %r"
                      % (mean, std * 2, params))
            print()

In [13]:
# Create and train XGBoost model
def create_and_train_xgb_model(X_samples, y_samples):
    
    # Split data into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(X_samples,
                                                        y_samples,
                                                        test_size=0.2,
                                                        random_state=301)
    
    # with future cases and npis are negatively correlated.
    model = xgb.XGBRegressor(
        colsample_bytree=0.3, 
        gamma=5,
        learning_rate=0.01,
        max_depth=10,
        n_estimators=1000,
        reg_alpha=50
    )

    # Fit model
    model.fit(X_train, y_train)
    
    # Evaluate model
    train_preds = model.predict(X_train)
    train_preds = np.maximum(train_preds, 0) # Don't predict negative cases
    print('Train MAE:', mae(train_preds, y_train))

    test_preds = model.predict(X_test)
    test_preds = np.maximum(test_preds, 0) # Don't predict negative cases
    print('Test MAE:', mae(test_preds, y_test))
    
    return model

In [ ]:
# Train a model for each day ahead we want to predict

# Set number of past days to use to make predictions
nb_lookback_days = 30

# Maximum number of days ahead we want to predict
max_days_ahead = 31

models = {}
for nb_days_ahead in range(max_days_ahead):
    print('Days ahead predicting:', nb_days_ahead)
    X_samples, y_samples = create_training_data(nb_days_ahead, nb_lookback_days, df, cases_col, npi_cols)
    X_samples
    y_samples
    # model = create_and_train_xgb_model(X_samples, y_samples)
    models[nb_days_ahead] = create_and_train_xgb_model(X_samples, y_samples)

Days ahead predicting: 0


In [ ]:
%matplotlib inline
# Inspect the learned feature coefficients for each model
# to see what features they're paying attention to.
for nb_days_ahead in range(max_days_ahead):
    print('Model for predicting {} days ahead'.format(nb_days_ahead))

    model = models[nb_days_ahead]
    plot_importance(model)

In [ ]:
# Save models to file
with open('models.pkl', 'wb') as models_file:
    pickle.dump(models, models_file)

## Evaluation

Now that the predictor has been trained and saved, this section contains the functionality for evaluating it on sample evaluation data.

First, a sample evaluation data set is created of the form that is given to the predictor.

Second, the predictor is evaluated on this data set, and a resulting predictions file is produced.

In [ ]:
### Create sample evaluation data

In [ ]:
# Create hypothetical evaluation data
nb_eval_days = 31
test_df = pd.read_csv(URL, 
                      parse_dates=['Date'],
                      encoding="ISO-8859-1",
                      error_bad_lines=False)

# Pull out relevant evaluation days
test_df = test_df[(test_df.Date > HYPOTHETICAL_SUBMISSION_DATE) & \
                  (test_df.Date <= HYPOTHETICAL_SUBMISSION_DATE + nb_eval_days)]

# Only include columns we would see during evaluation
test_df = test_df[['CountryName', 'RegionName', 'Date'] + npi_cols]

# Fill any missing NPIs by assuming they are the same as previous day
for npi_col in npi_cols:
    test_df.update(test_df.groupby(['CountryName', 'RegionName'])[npi_col].ffill().fillna(0))

In [ ]:
# test_df is now in the form of input to a predictor during evaluation
test_df


In [ ]:
def predict(start_date: str, end_date: str, path_to_ips_file: str):
    """
    Generates a file with daily new cases predictions for the given countries, regions and npis, between
    start_date and end_date, included.
    :param start_date: day from which to start making predictions, as a string, format YYYY-MM-DDD
    :param end_date: day on which to stop making predictions, as a string, format YYYY-MM-DDD
    :param path_to_ips_file: path to a csv file containing the intervention plans between start_date and end_date
    :return: Nothing. Saves a csv file called 'start_date_end_date.csv'
    with columns "CountryName,RegionName,Date,PredictedDailyNewCases"
    """
    
    # Add RegionID column that combines CountryName and RegionName for easier manipulation of data\n",
    test_df['GeoID'] = test_df['CountryName'] + '__' + test_df['RegionName'].astype(str)

    # Copy the test data frame
    pred_df = test_df[id_cols].copy()
    # Keep only the requested prediction period.
    # Note: this period *might* be in the future, and pred_df doesn't necessarily contain the requested rows
    pred_df = pred_df[(pred_df.Date >= start_date) & (pred_df.Date <= end_date)]

    # Load historical data to use in making predictions in the same way 
    # df is loaded above to make training data (just copy the reference here for simplicity)
    hist_df = df
    
        # Load models
    with open('models.pkl', 'rb') as models_file:
        models = pickle.load(models_file)
        
    # Make predictions for each country,region pair

    geo_pred_dfs = []
    for g in test_df.GeoID.unique():
        print('\nPredicting for', g)

        # Pull out all relevant data for country c
        hist_gdf = hist_df[hist_df.GeoID == g]
        test_gdf = test_df[test_df.GeoID == g]
        X_cases = np.array(hist_gdf[cases_col])[-nb_lookback_days:]
        X_hist_npis = np.array(hist_gdf[npi_cols])[-nb_lookback_days:]
        future_npi_data = np.array(test_gdf[npi_cols])

        # Make prediction for each day
        geo_preds = []
        nb_days_to_predict = len(future_npi_data)
        for days_ahead in range(nb_days_to_predict):

            # Prepare data
            X_future_npis = future_npi_data[:days_ahead]
            X_npis = np.concatenate([X_hist_npis, X_future_npis])
            X = np.concatenate([X_cases.flatten(),
                                X_npis.flatten()])

            # Grab the right model
            model = models[days_ahead]

            # Make the prediction (reshape so that sklearn is happy)
            pred = model.predict(X.reshape(1, -1))[0]
            pred = max(0, pred)
            geo_preds.append(pred)

        # Create geo_pred_df with pred column
        geo_pred_df = test_gdf[id_cols].copy()
        geo_pred_df['PredictedDailyNewCases'] = geo_preds
        geo_pred_dfs.append(geo_pred_df)

    # Combine all predictions into a single dataframe
    pred_df = pd.concat(geo_pred_dfs)
    
    # Drop GeoID column to match expected output format
    pred_df = pred_df.drop(columns=['GeoID'])
    pred_df
    
    # Write predictions to csv
    # Save to expected file name
    output_file_name = start_date + "_" + end_date + ".csv"
    pred_df.to_csv(output_file_name, index=None)
    print(f"Predictions saved to {output_file_name}")


In [ ]:
start_date = "2020-08-01"
end_date = "2020-08-31"

In [ ]:
predict(start_date, end_date, path_to_ips_file="../2020-08-01_2020-08-31_npis_example.csv")

In [ ]:
# Check that predictions are written correctly
!head 2020-08-01_2020-08-31.csv

In [ ]:
# Check the pediction file is valid
from validation.validation import validate_submission

errors = validate_submission(start_date, end_date, "2020-08-01_2020-08-31.csv")
if errors:
    for error in errors:
        print(error)
else:
    print("All good!")